Autorzy: Paulina Szymanek, Tomasz Kurcaba IiAD 3 rok

In [1]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns

import plotly.express as px
from wordcloud import WordCloud, STOPWORDS

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'plotly'

In [ ]:
!pip install wordcloud

In [ ]:
#load data
dataframe=pd.read_csv("trump_insult_tweets_2014_to_2021.csv", index_col=0)
dataframe

Jako postać polityczna Donald J. Trump używał Twittera do wychwalania, nagabywania, zabawiania, lobbowania, ustalania własnej wersji wydarzeń – i, co być może najbardziej znaczące, do wzmacniania swojej pogardy. Ten dataset dokumentuje ataki werbalne, które Trump opublikował na Twitterze, od ogłoszenia swojej kandydatury w czerwcu 2015 r. do 8 stycznia, kiedy Twitter na stałe go zabronił.
Źródło: Archiwum Trumpa na Twitterze
Ten zbiór danych został pobrany z The New York Times.

Zbiór ten zawiera kolumny: date, target, insult, tweet   
date - data tweeta   
target - cel, osoba/tematyka w jaką wymierzony był tweet   
insult - użyta "zniewaga"   
tweet - cały tweet  



In [ ]:
twitt_value_counts =dataframe['target'].value_counts()
twitt_value_counts.head()

Widzimy, że Donald Trump najczęściej kierował tweety w stronę szeroko pojętych mediów, demokratów i hillary clinton. Wysoko jest jego kolejny rywal czyli Joe Biden.

In [ ]:
regex = re.compile(r'@\w+')

dest_list = []
temp = ''
for val in dataframe['tweet']:
    temp = regex.findall(str(val))
    temp_2 = ''
    if temp:
        if len(temp) > 1:
            temp_2 = ' '.join(temp)
        else: 
            temp_2 = temp[0]
    else:
        temp_2 = 'None'
    dest_list.append(temp_2)

for i in range(10):
    print(dest_list[i])

In [ ]:
dest_array = np.array(dest_list)
dataframe['mentions']= pd.Series(dest_array)
dataframe['mentions'].value_counts().head(50)

In [ ]:
best = ['@nytimes', '@CNN', '@FoxNews', '@foxandfriends', '@JebBush'] 
counts = [236, 179, 172, 75, 43] # seats in assembly
explode=(0.2, 0.2, 0.2, 0.2, 0.2) # distance from each other
pie=plt.pie(counts,
        explode=explode,
        labels=best,
        autopct='%.1f%%',
        shadow=True,
        startangle=45
)
plt.axis('equal')
plt.title('Graph of top 5 mentions')
plt.legend(pie[0], best, loc=2)
plt.show()

In [ ]:
stopwords = set(STOPWORDS)
text = dataframe.tweet.to_csv()
wc  = WordCloud( background_color='white', stopwords=stopwords)
wc.generate(text)
plt.figure(figsize=(14,14))
plt.imshow(wc)
plt.show()

Poglądowo spójrzymy na powyższy wykres który pokazuje występowanie słów/pojedynczych wyrazów w badanym zbiorze.

Najczęstsze wzmianki w tweetach prezydenta Trump to The New York Times, Jeb Bush (był gubernator Florydy), Fox & Friends (talkshow), Fox News Channel(stacja informacyjna), CNN(telewizja informacyjna).

Celem niniejszego projektu będzie określenie podobieństwa między tweetem, który jest oczyszczony ze wszystkich znaków dodatkowych z wejściowych tweetem. Zostanie to zrobione przy użyciu macierzy funkcji TF-IDF.   
TF to pozwala określić częstotliwość występowania danej frazy w konkretnym dokumencie. Jego zakres obejmuje wyłącznie jeden dokument, który ma być badany. Wartość ta jest wprost proporcjonalna do częstotliwości występowania wyrazu.Aby obliczyć TF, należy podzielić liczbę wystąpień frazy w dokumencie przez liczbę znajdujących się w nim wszystkich słów.   
IDF natomiast  pozwala sprawdzić, jak często dany termin występuje we wszystkich dokumentach badanego korpusu językowego. W zależności od potrzeb może być chociażby TOP10 wyników wyszukiwania, zbiór artykułów naukowych z danej dziedziny, konkretny zbiór tekstów, itp. Jest to odwrotna częstotliwość wyrażenia, dlatego też im częściej dany wyraz pojawia się w korpusie, tym wynik IDF będzie niższy. We wzorze użyty jest logarytm.   
Używaną przez nas metryką będzie podobieństwo cosinusowe. Metryka ta mierzy podobieństwo między dwoma wektorami wewnętrznej przestrzeni iloczynu. Jest mierzony przez cosinus kąta między dwoma wektorami i określa, czy dwa wektory wskazują mniej więcej w tym samym kierunku. Jest często używany do pomiaru podobieństwa dokumentów w analizie tekstu. Dokument może być reprezentowany przez tysiące atrybutów, z których każdy rejestruje częstotliwość występowania określonego słowa (takiego jak słowo kluczowe) lub frazy w dokumencie. Tak więc każdy dokument jest obiektem reprezentowanym przez tak zwany wektor częstotliwości.

In [ ]:
#global variables
STOP_WORDS=set(nltk.corpus.stopwords.words('english'))
PORTER=nltk.stem.porter.PorterStemmer()
LEMMATIZER=nltk.stem.wordnet.WordNetLemmatizer()

In [ ]:
#function for cleaning string
def clean_string(string):
    words_split=nltk.word_tokenize(string)
    words_split=[re.sub(r"[^a-zA-Z0-9]+",'',word) for word in words_split]
    words_split=list(filter(None,words_split))
    words_split=[word.lower() for word in words_split]
    words_clean=[word for word in words_split if word not in STOP_WORDS]
    words_stemmed=[PORTER.stem(word) for word in words_clean]
    output=' '.join(words_stemmed)
    return output

In [ ]:
def basic_clean_string(string):
    words_split=nltk.word_tokenize(string)
    words_split=[re.sub(r"[^a-zA-Z0-9]+",'',word) for word in words_split]
    words_split=list(filter(None,words_split))
    words_split=[word.lower() for word in words_split]
    words_clean=[word for word in words_split if word not in STOP_WORDS]
    words_stemmed=[LEMMATIZER.lemmatize(word) for word in words_clean]
    output=' '.join(words_stemmed)
    return output

In [ ]:
#clean columns
dataframe['insult_cleaned']=dataframe['insult'].apply(lambda x: clean_string(x))
dataframe['tweet_cleaned']=dataframe['tweet'].apply(lambda x: clean_string(x))
dataframe

In [ ]:
len(dataframe['insult_cleaned'].unique())
len(dataframe['tweet_cleaned'].unique())

In [ ]:
insults = dataframe['insult_cleaned'].unique() #list of unique insults
insults

In [ ]:
dataframeNew = dataframe.groupby(['tweet_cleaned','target','tweet'])['insult_cleaned'].apply(list).reset_index(name='insult_cleaned')
dataframeNew

In [ ]:
vectorizer=TfidfVectorizer(max_features=1000)

In [ ]:
df_tfidf=vectorizer.fit_transform(dataframeNew['tweet_cleaned']).todense()

In [ ]:
#calculating similarity measure
cosine_similarity_measure=cosine_similarity(df_tfidf)

In [ ]:
#creating dictionary with indexes of tweet that contain that insult
key_indexes_dict={}
for insult in insults:
    key_indexes_dict[insult]=[]
    for idx, row in enumerate(dataframeNew['insult_cleaned']):
        if insult in row:
                key_indexes_dict[insult].append(idx)

In [ ]:
key_indexes_dict

In [ ]:
#adding dummy row for case when no tweet has the same set of insults
dataframeNew.loc[dataframeNew.shape[0]-1,:]=['No match found','No match found','No match found','No match found']

In [ ]:
guessed_text=[]
sim_measure=[]
target_bonus=0.1 #bonus for the same tweet target

for text_idx, text_insult in enumerate(dataframeNew['insult_cleaned']):
    if text_idx == dataframeNew.last_valid_index():
        guessed_text.append(len(dataframeNew["tweet"])-1)
        sim_measure.append(0)
        break;
    rows_indexes=set(key_indexes_dict[text_insult[0]])
    if len(text_insult)>1: #if more than one insult
        for elem_in_key in range(1,len(text_insult)-1):
            rows_indexes=rows_indexes&set(key_indexes_dict[text_insult[elem_in_key]])
        rows_indexes=list(rows_indexes)
        rows_indexes.sort()
    if rows_indexes: #matches found
        maximum_sim=0
        maximum_idx=0
        for row in rows_indexes:
            if row == text_idx:
                maximum_sim=0
                maximum_idx=dataframeNew.last_valid_index()
                continue;
            cos_sim=cosine_similarity_measure[text_idx,row]
            if dataframeNew.loc[row,'target']==dataframeNew.loc[text_idx,'target']:
                cos_sim+=target_bonus
            if cos_sim>maximum_sim:
                maximum_sim=cos_sim
                maximum_idx=row
        guessed_text.append(maximum_idx)
        sim_measure.append(maximum_sim)
    else: 
        guessed_text.append(len(dataframeNew["tweet"])-1)
        sim_measure.append(0)

dataframeNew['guessed_tweet']=dataframeNew.loc[guessed_text, 'tweet'].tolist()
dataframeNew['similarity']=sim_measure
dataframeNew

In [ ]:
count_vectorizer3 = CountVectorizer(ngram_range=(3,3), min_df=10)
count_matrix3 = count_vectorizer3.fit_transform(dataframeNew['tweet'].apply(lambda x: basic_clean_string(x))).todense()

counts3 = pd.DataFrame(count_matrix3, columns = count_vectorizer3.get_feature_names())

In [ ]:
counts3.sum().sort_values(ascending=False)[0:10]

Najczęściej pojawiące się 3-gramy w badanym tekście. Możemy wywnioskować, że Donald Trump to bardzo wyrazista postać, która nie potrafi ugryźć się w język, nawet na twitterze.

In [ ]:
count_vectorizer2 = CountVectorizer(ngram_range=(2,2), min_df=10)
count_matrix2 = count_vectorizer2.fit_transform(dataframeNew['tweet'].apply(lambda x: basic_clean_string(x))).todense()

counts2 = pd.DataFrame(count_matrix2, columns = count_vectorizer2.get_feature_names())
counts2.sum().sort_values(ascending=False)[0:10]

Podobne wnioski dla 2-gramów.

In [ ]:
count_vectorizer1 = CountVectorizer( min_df=50)
count_matrix1 = count_vectorizer1.fit_transform(dataframeNew['tweet'].apply(lambda x: basic_clean_string(x))).todense()

counts1 = pd.DataFrame(count_matrix1, columns = count_vectorizer1.get_feature_names())
counts1.sum().sort_values(ascending=False)[0:10]


In [ ]:
plt.figure(figsize=(16, 16))
plt.bar(counts1.sum().sort_values(ascending=False)[0:15].index,counts1.sum().sort_values(ascending=False)[0:15])


In [ ]:
plt.figure(figsize=(16, 16))
plt.bar(counts2.sum().sort_values(ascending=False)[0:12].index,counts2.sum().sort_values(ascending=False)[0:12])


In [ ]:
plt.figure(figsize=(16, 16))
plt.bar(counts3.sum().sort_values(ascending=False)[0:8].index,counts3.sum().sort_values(ascending=False)[0:8])


In [ ]:
dataframeNew['similarity'].describe()

In [ ]:
dataframeNew['similarity'].hist()

Widzimy, że nasz model przyporządkował bardzo dużo tweetów jako niepodobnych, mimo to sporo jest takich mających wartości z przedziału 0.2-0.4 oraz w okolicach wartości 1.